In [5]:
import requests
import time

def get_public_ip():
    response = requests.get('https://api.ipify.org').text
    return response

interval = 5  # Interval in seconds
while True:
    public_ip = get_public_ip()
    print(f'Public IPv4: {public_ip}')
    time.sleep(interval)

Public IPv4: 82.64.76.148


KeyboardInterrupt: 

# Code pour créer nos 5 tables

In [7]:
import psycopg2

def create_tables():

    # Initialisation de la variable de connexion
    conn = None

    # Paramètres de connexion
    params = {
        'host': 'lm1672121-001.eu.clouddb.ovh.net',
        'port': 35602,
        'user': 'admin',
        'password': 'Hermell15C',
        'dbname': 'Hermel'
    }

    try:
        # Connexion à la base de données PostgreSQL
        conn = psycopg2.connect(**params)

        # Création d'un nouvel objet cursor
        cur = conn.cursor()

        # Requêtes SQL pour créer les tables
        queries = [
            """
            CREATE TABLE IF NOT EXISTS Users (
                id SERIAL PRIMARY KEY,
                username VARCHAR(50) UNIQUE,
                email VARCHAR(50) UNIQUE,
                password_hash VARCHAR(50),
                created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
                updated_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
            );
            """,
            """
            CREATE TABLE IF NOT EXISTS Campaigns (
                id SERIAL PRIMARY KEY,
                name VARCHAR(50),
                user_id INT REFERENCES Users(id),
                created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
                updated_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
            );
            """,
            """
            CREATE TABLE IF NOT EXISTS Emails (
                id SERIAL PRIMARY KEY,
                subject VARCHAR(50),
                body TEXT,
                campaign_id INT REFERENCES Campaigns(id),
                created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
                updated_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
            );
            """,
            """
            CREATE TABLE IF NOT EXISTS Contacts (
                id SERIAL PRIMARY KEY,
                email VARCHAR(50),
                first_name VARCHAR(50),
                last_name VARCHAR(50),
                phone_number VARCHAR(50),
                user_id INT REFERENCES Users(id),
                created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
                updated_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
            );
            """,
            """
            CREATE TABLE IF NOT EXISTS Campaigns_Contacts (
                campaign_id INT REFERENCES Campaigns(id),
                contact_id INT REFERENCES Contacts(id),
                PRIMARY KEY (campaign_id, contact_id)
            );
            """
        ]

        # Exécution des requêtes pour créer les tables
        for query in queries:
            cur.execute(query)

        # Fermeture de la communication avec la base de données
        cur.close()

        # Commit des changements
        conn.commit()

    except Exception as e:
        print(f"Une erreur est survenue : {e}")

    finally:
        if conn:
            # Fermeture de la connexion
            conn.close()

if __name__ == '__main__':
    create_tables()


# Code pour voir quels tables nous avons dans la DB

In [8]:
import psycopg2

def list_tables():

    # Paramètres de connexion
    params = {
        'host': 'lm1672121-001.eu.clouddb.ovh.net',
        'port': 35602,  # Remplacez par le bon port si nécessaire
        'user': 'admin',
        'password': 'Hermell15C',
        'dbname': 'Hermel'
    }

    # Initialisation de la variable de connexion
    conn = None

    try:
        # Connexion à la base de données PostgreSQL
        conn = psycopg2.connect(**params)

        # Création d'un nouvel objet cursor
        cur = conn.cursor()

        # Exécution de la requête SQL pour lister toutes les tables
        cur.execute("""
            SELECT table_name
            FROM information_schema.tables
            WHERE table_schema = 'public'
        """)

        # Récupération des résultats
        tables = cur.fetchall()

        # Affichage des tables
        for table in tables:
            print("Table:", table[0])

        # Fermeture de la communication avec la base de données
        cur.close()

    except Exception as e:
        print(f"Une erreur est survenue : {e}")

    finally:
        if conn:
            # Fermeture de la connexion
            conn.close()

if __name__ == '__main__':
    list_tables()


Table: users
Table: campaigns
Table: emails
Table: contacts
Table: campaigns_contacts


# CODE POUR IMPORTER DE LA DATA VIA SQL/.csv

In [13]:
import pandas as pd
import psycopg2
from sqlalchemy import create_engine

# Paramètres de connexion à la base de données
params = {
    'host': 'lm1672121-001.eu.clouddb.ovh.net',
    'port': 35602,  # Remplacez par le bon port si nécessaire
    'user': 'admin',
    'password': 'Hermell15C',
    'dbname': 'Hermel'
}

# Création d'une connexion SQLAlchemy
engine = create_engine(f"postgresql+psycopg2://{params['user']}:{params['password']}@{params['host']}:{params['port']}/{params['dbname']}")

# Lecture des fichiers CSV avec pandas
df_users = pd.read_csv('users.csv')
df_campaigns = pd.read_csv('campaigns.csv')
df_emails = pd.read_csv('emails.csv')
df_contacts = pd.read_csv('contacts.csv')
df_campaigns_contacts = pd.read_csv('campaigns_contacts.csv')

# Importation des données dans PostgreSQL
df_users.to_sql('users', engine, if_exists='append', index=False)
df_campaigns.to_sql('campaigns', engine, if_exists='append', index=False)
df_emails.to_sql('emails', engine, if_exists='append', index=False)
df_contacts.to_sql('contacts', engine, if_exists='append', index=False)
df_campaigns_contacts.to_sql('campaigns_contacts', engine, if_exists='append', index=False)


5